### Install packages and import

In [ ]:
import os # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

if code_dir_name not in str(Path.cwd()).split('/')[-1]:
    for _ in range(5):

        parent_path = str(Path.cwd().parents[_]).split('/')[-1]

        if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

            code_dir = str(Path.cwd().parents[_])

            if code_dir is not None:
                break
else:
    code_dir = str(Path.cwd())
sys.path.append(code_dir)

# %load_ext autoreload
# %autoreload 2


### Define some Funtions

In [ ]:
# Function to check for pop-ups
def popup(driver):

    # Signup popup
    try:
        signup_popup = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_id("popover-form-container")
        )
        if (signup_popup.is_displayed()) and (signup_popup.is_enabled()):
            # print('Signup popup detected.')
            close_signup_popup = driver.find_element_by_xpath(
                '//*[@id="popover-x"]/button'
            )
            close_signup_popup.click()
            # print('Signup popup dismissed.')
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "input_submit"))
            )
    except errors:

        # Accept Alert
        try:
            click_alert = driver.switch_to.alert()
            # print('Alert found.')
            click_alert.accept()
            # print('Alert accepted.')
            WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "input_submit"))
            )
        except errors:

            # Cookie popup
            try:
                cookie_popup = WebDriverWait(driver, 10).until(
                    lambda driver: driver.find_element_by_id(
                        "onetrust-accept-btn-handler"
                    )
                )
                if (cookie_popup.is_displayed()) and (cookie_popup.is_enabled()):
                    # print('Cookie detected.')
                    cookie_popup.click()
                    # print('Cookie accepted.')
                    WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.CLASS_NAME, "input_submit"))
                    )
            except errors:
                pass
            # Cookie popup
            except:
                try:
                    print(f"Cookie popup caused unexpected error: {sys.exc_info()[0]}")
                except (NoSuchWindowException, urllib3.exceptions.MaxRetryError):
                    pass
        # Accept Alert
        except:
            try:
                print(f"Alert caused unexpected error: {sys.exc_info()[0]}")
            except (NoSuchWindowException, urllib3.exceptions.MaxRetryError):
                pass
    # Signup popup
    except:
        try:
            print(f"Signup popup caused unexpected error: {sys.exc_info()[0]}")
        except (NoSuchWindowException, urllib3.exceptions.MaxRetryError):
            pass

    return


In [ ]:
# Find selected language and make sure it's English if available
def find_language(driver, soup, language):
    language_chooser_list = WebDriverWait(driver, 10).until(
        lambda driver: driver.find_elements_by_xpath(
            '//*[@id="filter-language"]/button'
        )
    )

    for language_chooser in language_chooser_list:
        if language_chooser.text == "Vacaturetaal":
            print("Language chooser found.")
            language_chooser.click()

            driver.execute_script(
                "arguments[0].setAttribute('aria-expanded', arguments[1])",
                language_chooser,
                "true",
            )

            drop_down = language_chooser.get_attribute("aria-expanded")

            if str(drop_down) == "true":
                print("Language dropdown menu activated.")

                # Get html of page with beautifulsoup4
                # html = driver.page_source
                # print('Feeding html driver to BeautifulSoup.')
                # soup = BeautifulSoup(html, 'lxml')

                language_dropdown = soup.find_all(
                    "ul", class_="filter-language-menu", id="filter-language-menu"
                )

                for language in language_dropdown:
                    language_english = xpath_soup(
                        language.find("a").find(
                            "span", class_="rbLabel", text=re.compile(str(language))
                        )
                    )
                    language_button = WebDriverWait(driver, 10).until(
                        lambda driver: driver.find_element_by_xpath(language_english)
                    )
                    language_button.click()
                    print(f"{str(language)} language chosen.")

            elif str(drop_down) == "false":
                print("Language dropdown menu NOT activated.")

                # Get html of page with beautifulsoup4
                # html = driver.page_source
                # print('Feeding html driver to BeautifulSoup.')
                # soup = BeautifulSoup(html, 'lxml')

                # soup = find_language(driver, soup, language)

        elif language_chooser.text != "Vacaturetaal":
            print(
                f"Language chooser NOT found. {language_chooser.text} chooser found instead."
            )

    return soup


In [ ]:
# Function to find page number
def find_page_number(driver, soup, page_counter):
    try:
        try:
            page_footer = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "pagination"))
            )
        except:
            print("No page footer found. Trying different method.")
            page_footer = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "secondRow"))
            )
            page_number_text = str(
                soup.find("div", class_="secondRow")
                .find("div", class_="searchCountContainer")
                .find("div", id="searchCountPages")
                .text
            )
            page_number_length = [
                int(s) for s in page_number_text.split() if s.isdigit()
            ][1]
            page_number = [int(s) for s in page_number_text.split() if s.isdigit()][0]
        else:
            page_number_list = soup.find("ul", class_="pagination-list").find_all("li")
            page_number_length = len(
                soup.find("ul", class_="pagination-list").find_all("li")
            )
            if int(page_number_length) == 1:
                print(f"Only 1 page found for {str(keyword)} jobs.")
                print(f"CURRENT PAGE NUMBER: 1.")
            elif int(page_number_length) != 1:
                print(
                    f"Total of {page_number_length} pages found for {str(keyword)} jobs."
                )
            try:
                page_number = int(
                    soup.find("ul", class_="pagination-list")
                    .find(attrs={"aria-current": "true"})
                    .text
                )
            except:
                page_number = [int(s) for s in page_number_text.split() if s.isdigit()][
                    0
                ]

    except:
        page_number = page_counter + 1
        page_number_length = 1
        print(
            f"Unexpected error getting page number: {sys.exc_info()[0]}. Page number set to page counter + 1: {page_number}"
        )

    print("-" * 20)
    print(f"CURRENT PAGE NUMBER: {page_number}.")
    print("-" * 20)

    return page_number_length, page_number


In [ ]:
# Collect individual job links
def get_each(
    driver,
    soup,
    main_window,
    window_before,
    keyword,
    keyword_file,
    site,
    jobs,
    job_soup,
    job_id,
    jobs_count,
    page_counter,
    page_number,
    page_number_length,
    save_path,
    json_file_name,
):

    # If results open in new window, make sure everything loads
    new_window = check_window(driver, main_window, window_before)

    # Get html of page with beautifulsoup4
    # html = driver.page_source
    # print('Feeding html driver to BeautifulSoup.')
    # soup = BeautifulSoup(html, 'lxml')

    # Get job id, url and age
    print("-" * 20)
    print(f"Job ID: {job_id}")
    print("-" * 20)
    job_url = "https://nl.indeed.com/vacature-bekijken" + str(
        job_soup.find("h2", class_="title").find("a")["href"]
    ).replace("/rc/clk", "")
    print(f"Job URL: {job_url}")
    print("-" * 20)
    job_age = (
        soup.find("div", class_="result-link-bar")
        .find("span", class_="date")
        .find(string=re.compile(r".*"))
    )
    for s in job_age:
        job_age_number = int(s) if s.isdigit() else job_age
    print(f"Job Age: {job_age}")
    print("-" * 20)

    #########################################################
    #########################################################
    ####################### TAB INFO #######################
    #########################################################
    #########################################################

    # Get main data
    # try:
    (
        job_title,
        job_description,
        company_name,
        location,
        rating,
        company_url,
        emp_type,
    ) = get_main(driver, soup)

    # In case there is no data, -1 for all variables
    # except NoSuchElementException:
    #     print(f'No data collected: {sys.exc_info()[0]}.')
    #     job_title, job_description, company_name, location, rating, company_url, emp_type = assign_all(7, -1)

    # Save to dict
    print("Saving to dict...")
    today = datetime.date.today()
    jobs.append(
        {
            "Search Keyword": str(keyword),
            "Platform": site,
            "Job ID": job_id,
            "Job Title": str(job_title),
            "Company Name": str(company_name),
            "Location": str(location),
            "Job Description": str(job_description),
            "Rating": float(rating),
            "Employment Type": str(emp_type),
            "Company URL": company_url,
            "Job URL": job_url,
            "Job Age": job_age,
            "Job Age Number": job_age_number,
            "Collection Date": (today.strftime("%Y-%m-%d")),
        }
    )

    # Save dict as json file
    jobs = remove_dupe_dicts(jobs)
    print(f"Progress for {keyword} job ads (page {page_number}): {len(jobs)}")
    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)

    # Save page counter
    if page_counter > 0:
        with open(
            save_path + f"{site.lower()}_page_counter_{keyword_file}.txt", "w"
        ) as f:
            f.write(str(page_counter))

    # Check current window and go back if required
    check_window_back(driver, main_window, window_before, new_window)

    return jobs


In [ ]:
# Function to get main job data
def get_main(driver, soup):

    pattern = re.compile(r"[^A-Za-z0-9, /]+")
    # WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id=\"vjs-container\"]')))

    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    try:
        container = soup.find("div", id="vjs-container")
        try:
            job_title = str(
                container.find("div", id="vjs-jobinfo")
                .find("div", id="vjs-jobtitle")
                .text
            )
        except NoSuchElementException:
            print(f"No job title found.")
            job_title = -1
        except errors:
            print(f"Unexpected error getting job title: {sys.exc_info()[0]}.")
            job_title = -1
        else:
            print("-" * 20)
            print(f"Job Title: {job_title}")
            print("-" * 20)

        try:
            job_desc = container.find("div", id="vjs-content").find_all(
                "div", id="vjs-desc"
            )
            job_description = ""
            for job_d in job_desc:
                job_description += str(job_d.text).replace("\\uf0b7", "\\n")
        except NoSuchElementException:
            print(f"No job description found.")
            job_description = -1
        except errors:
            print(f"Unexpected error getting job description: {sys.exc_info()[0]}.")
            job_description = -1
        else:
            print("-" * 20)
            print(f"Job Description: {job_description[:50]}")
            print("-" * 20)

        try:
            company_name = pattern.sub(
                "",
                str(
                    container.find("div", id="vjs-jobinfo")
                    .find("span", id="vjs-cn")
                    .text
                ).strip(),
            )
        except NoSuchElementException:
            print(f"No job company name found.")
            company_name = -1
        except errors:
            print(f"Unexpected error getting company name: {sys.exc_info()[0]}.")
            company_name = -1
        else:
            print("-" * 20)
            print(f"Company Name: {company_name}")
            print("-" * 20)

        try:
            location = pattern.sub(
                "", str(container.find("span", id="vjs-loc").text).strip()
            )
        except NoSuchElementException:
            print(f"No job company location found.")
            location = -1
        except errors:
            print(f"Unexpected error getting company location: {sys.exc_info()[0]}.")
            location = -1
        else:
            print("-" * 20)
            print(f"Company Location: {location}")
            print("-" * 20)

        try:
            company_url = "https://indeed.nl" + str(
                container.find("div", id="vjs-header")
                .find("div", id="apply-button-container")
                .find("a")["href"]
            )
        except NoSuchElementException:
            print(f"No job company url found.")
            company_url = -1
        except errors:
            print(f"Unexpected error getting company url: {sys.exc_info()[0]}.")
            company_url = -1
        else:
            print("-" * 20)
            print(f"Company URL: {company_url}")
            print("-" * 20)

    except errors:
        print(f"Unexpected error getting container: {sys.exc_info()[0]}.")
        job_title, job_description, company_name, location, company_url = assign_all(
            5, -1
        )

    try:
        premium = (
            driver.find_element_by_xpath(
                '//*[@id="p_81291054cf8540ca"]/table/tbody/tr/td/span[2]/font/font'
            )
        ).text

        try:
            rating = float(
                (
                    driver.find_element_by_xpath(
                        '//*[@id="vjs-jobinfo"]/div[2]/a/span[1]'
                    )
                ).text
            )
        except NoSuchElementException:
            print(f"No job rating found.")
            rating = -1
        except errors:
            print(f"Unexpected error getting rating: {sys.exc_info()[0]}.")
            rating = -1
        else:
            print("-" * 20)
            print(f"Rating: {rating}")
            print("-" * 20)

        try:
            print("-" * 20)
            emp_type = (
                driver.find_element_by_xpath('//*[@id="vjs-jobinfo"]/div[3]/span')
            ).text
            print(f"Employment Type: {emp_type}")
            print("-" * 20)
        except NoSuchElementException:
            print(f"No job employment type found.")
            emp_type = -1
        except errors:
            print(f"Unexpected error getting employment type: {sys.exc_info()[0]}.")
            emp_type = -1
        else:
            print("-" * 20)
            print(f"Employment Type: {emp_type}")
            print("-" * 20)

    except NoSuchElementException:
        print(f"No premium tag.")
        rating, emp_type = assign_all(2, -1)

    except:
        print(
            f"Unexpected error getting ratings and employment type: {sys.exc_info()[0]}."
        )
        rating, emp_type = assign_all(2, -1)

    # Check if collection successful
    imp_collected = [job_title, job_description, company_name, location]
    if all((var != int(-1)) or (var != str("-1")) for var in imp_collected):
        print("Main data collected successfully.")

    elif any((var == int(-1)) or (var == str("-1")) for var in imp_collected):
        print(
            f"Main data NOT collected successfully. Sleeping for 10 seconds then trying again."
        )
        time.sleep(10)
        try:
            (
                job_title,
                job_description,
                company_name,
                location,
                rating,
                company_url,
                emp_type,
            ) = get_main(driver, soup)
        except:
            print(
                f"Data NOT collected successfully. Not attempting again: {sys.exc_info()[0]}."
            )

    return (
        job_title,
        job_description,
        company_name,
        location,
        rating,
        company_url,
        emp_type,
    )


In [ ]:
# Click on next page button
def next_page(
    driver,
    soup,
    keyword,
    keyword_file,
    site,
    main_window,
    window_before,
    counter,
    page_counter,
    page_number,
    page_number_length,
    jobs,
    df_old_jobs,
    save_path,
    json_file_name,
):

    # Get html of page with beautifulsoup4
    # html = driver.page_source
    # print('Feeding html driver to BeautifulSoup.')
    # soup = BeautifulSoup(html, 'lxml')

    try:
        page_footer = WebDriverWait(driver, 10).until(
            lambda driver: driver.find_element_by_class_name("pagination")
        )
    except:
        done = True
        print("No page footer found.")
    else:

        while page_footer.is_enabled():

            page_number = int(
                soup.find("ul", class_="pagination-list")
                .find(attrs={"aria-current": "true"})
                .text
            )
            page_number_for_next = soup.find("ul", class_="pagination-list").find(
                attrs={"aria-current": "true"}
            )
            page_number_next = page_number_for_next.find_next("li")

            try:
                if page_number_next.find("a").find("span")["class"][0] == "pn":
                    page_number_next_text = page_number_next.find("a")["aria-label"]
                    print(
                        f"More page buttons found. Next page number: {page_number_next_text}."
                    )
                    done = False

                    try:
                        # Determine current page number
                        page_number_for_next = soup.find(
                            "ul", class_="pagination-list"
                        ).find(attrs={"aria-current": "true"})
                        page_number_next = page_number_for_next.find_next("li")
                    except:
                        done = True
                        print("No next page button (Volgende) found.")
                        break
                    else:
                        try:
                            page_number_next_text = int(
                                page_number_next.find("a")["aria-label"]
                            )
                            page_counter = page_number
                            if page_counter > 0:
                                with open(
                                    save_path
                                    + f"{site.lower()}_page_counter_{keyword_file}.txt",
                                    "w",
                                ) as f:
                                    f.write(str(page_counter))
                            (
                                jobs,
                                jobs_count,
                                df_jobs,
                                counter,
                                page_counter,
                                done,
                            ) = get_jobs(
                                keyword.lower(),
                                keyword_url,
                                site,
                                language,
                                driver,
                                counter,
                                page_counter,
                                main_window,
                                df_old_jobs,
                                jobs,
                                save_path.lower(),
                                json_file_name.lower(),
                                done,
                            )
                        except:
                            # Get html of page with beautifulsoup4
                            # html = driver.page_source
                            # print('Feeding html driver to BeautifulSoup.')
                            # soup = BeautifulSoup(html, 'lxml')

                            page_number_length = len(
                                soup.find("ul", class_="pagination-list").find_all("li")
                            )
                            page_number = int(
                                soup.find("ul", class_="pagination-list")
                                .find(attrs={"aria-current": "true"})
                                .text
                            )
                            page_counter = page_number
                            print(f"Last page reached at page {int(page_number)}.")
                            done = True
                            break
                        else:
                            if int(page_number) == int(page_number_next_text):
                                page_number = int(
                                    soup.find("ul", class_="pagination-list")
                                    .find(attrs={"aria-current": "true"})
                                    .text
                                )
                                print(
                                    f"Page successfully changed to page {int(page_number)}"
                                )
                            elif int(page_number) != int(page_number_next_text):
                                page_number = int(
                                    soup.find("ul", class_="pagination-list")
                                    .find(attrs={"aria-current": "true"})
                                    .text
                                )
                                print(
                                    f"Page not successfully changed to page {int(page_number_next_text)}. Current page {int(page_number)}."
                                )

                elif page_number_next.find("a").find("span")["class"][0] != "pn":
                    done = True
                    print(f"Last page reached at page {int(page_number)}.")
                    break
                break

            except:
                done = True
                print("No more page buttons found.")
                break
            break
        else:
            done = True
            print("No page footer found.")

    # Save page counter
    if page_counter > 0:
        with open(
            save_path + f"{site.lower()}_page_counter_{keyword_file}.txt", "w"
        ) as f:
            f.write(str(page_counter))

    return int(page_number), jobs, counter, page_counter, done


In [ ]:
def get_page_jobs(
    driver,
    soup,
    keyword,
    keyword_file,
    site,
    main_window,
    window_before,
    counter,
    page_counter,
    page_number,
    page_number_length,
    jobs,
    df_old_jobs,
    save_path,
    json_file_name,
    done,
):

    # Find job button next siblings with bs4
    print("Finding job links in page.")

    # Get html of page with beautifulsoup4
    # html = driver.page_source
    # print('Feeding html driver to BeautifulSoup.')
    # soup = BeautifulSoup(html, 'lxml')

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "resultsCol"))
    )

    jobs_on_page = soup.find_all(
        "div",
        class_=[
            "jobsearch-SerpJobCard unifiedRow row result clickcard vjs-highlight",
            "jobsearch-SerpJobCard unifiedRow row result clickcard",
        ],
    )
    jobs_count = len(jobs_on_page)

    # if int(jobs_count) != 0:
    print(
        f"{int(jobs_count)} job(s) found on page {page_number} out of {page_number_length} page(s)."
    )
    print(f"Progress for {keyword} job ads (page {page_number}): {len(jobs)}")

    # Get html of page with beautifulsoup4
    # html = driver.page_source
    # print('Feeding html driver to BeautifulSoup.')
    # soup = BeautifulSoup(html, 'lxml')

    for job_soup in jobs_on_page:
        print("-" * 20)
        print(
            f"JOB NUMBER: {int(jobs_on_page.index(job_soup)) + 1} OUT OF {int(jobs_count)} ON PAGE {page_number} OUT OF {page_number_length}."
        )
        page_counter = page_number - 1
        if page_counter > 0:
            with open(
                save_path + f"{site.lower()}_page_counter_{keyword_file}.txt", "w"
            ) as f:
                f.write(str(page_counter))
        print("-" * 20)
        job_xpath = xpath_soup(job_soup)
        job_button = WebDriverWait(
            driver, 10, ignored_exceptions=StaleElementReferenceException
        ).until(lambda driver: driver.find_element_by_xpath(job_xpath))

        try:
            job_button.click()
        except ElementNotInteractableException:
            print(
                f"Click did not work. Trying again to get job soup for {str(keyword)}."
            )
            # Get html of page with beautifulsoup4
            html = driver.page_source
            print("Feeding html driver to BeautifulSoup.")
            soup = BeautifulSoup(html, "lxml")

        # Get job id
        job_id = str(job_soup["id"])

        # Check if already collected
        jobs, job_present = id_check(
            driver,
            main_window,
            window_before,
            keyword,
            site,
            df_old_jobs,
            jobs,
            job_soup,
            job_id,
            jobs_count,
            save_path,
            json_file_name,
        )
        if job_present is False:
            jobs = get_each(
                driver,
                soup,
                main_window,
                window_before,
                keyword,
                keyword_file,
                site,
                jobs,
                job_soup,
                job_id,
                jobs_count,
                page_counter,
                page_number,
                page_number_length,
                save_path,
                json_file_name,
            )
        elif job_present is True:
            print(f"Progress for {keyword} job ads (page {page_number}): {len(jobs)}")
    # elif int(jobs_count) == 0:
    #     print(f'No jobs results found for {str(keyword)}. No error detected. Moving on to next search.')

    # End of page
    print("-" * 20)
    print(f"REACHED END OF JOBS FOR PAGE: {page_number}.")
    print("-" * 20)

    # Save page counter
    if page_counter > 0:
        with open(
            save_path + f"{site.lower()}_page_counter_{keyword_file}.txt", "w"
        ) as f:
            f.write(str(page_counter))

    # Save dict as json file
    jobs = remove_dupe_dicts(jobs)
    print(f"Progress for {keyword} job ads (page {page_number}): {len(jobs)}")
    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)

    # Find next page
    page_number, jobs, counter, page_counter, done = next_page(
        driver,
        soup,
        keyword,
        keyword_file,
        site,
        main_window,
        window_before,
        counter,
        page_counter,
        page_number,
        page_number_length,
        jobs,
        df_old_jobs,
        save_path,
        json_file_name,
    )

    # Save the old page counter
    if page_counter > 0:
        with open(
            save_path + f"{site.lower()}_page_counter_{keyword_file}.txt", "w"
        ) as f:
            f.write(str(page_counter))

    print(f"End of data collection for {keyword}.\nGot {len(jobs)} jobs.")

    try:
        page_number_length = len(
            soup.find("ul", class_="pagination-list").find_all("li")
        )
        page_number = int(
            soup.find("ul", class_="pagination-list")
            .find(attrs={"aria-current": "true"})
            .text
        )
    except:
        page_number = counter

    # Save dict as json file
    done = True
    jobs = remove_dupe_dicts(jobs)
    jobs_count = len(jobs)
    print("-" * 20)
    try:
        print(f"Progress for {keyword} job ads (page {int(page_number)}): {len(jobs)}")
    except:
        pass
    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    print("-" * 20)
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)

    return jobs, jobs_count, counter, page_counter, int(page_number), done


### Get-jobs function

In [ ]:
# MAIN FUNCTION TO GET JOB ADS
def get_jobs(
    driver,
    keyword,
    keyword_url,
    keyword_file,
    site,
    language,
    counter,
    page_counter,
    main_window,
    df_old_jobs,
    jobs,
    save_path,
    json_file_name,
    df_file_name,
    done,
):

    #     if done is False:
    url = f"https://nl.indeed.com/jobs?q={keyword_url}&l=Noord-Holland&start={page_counter}0"

    # Go to page and establish main windows
    print(f"Going to url: {url}")
    driver.get(url)
    window_before = driver.window_handles[0]
    print(f"First Window: {window_before}")
    print(f"Current page title: {driver.title}")

    if "Captcha" in str(driver.title):
        print("!" * 30, "CAPTCHA FOUND", "!" * 30)
        act_cool(5, 500)
        ! openpyn nl
        ! funckymonkey89
        # Get jobs
        jobs, jobs_count, df_jobs, counter, page_counter, done = get_jobs(
            keyword.lower(),
            keyword_url,
            site,
            language,
            driver,
            counter,
            page_counter,
            main_window,
            df_old_jobs,
            jobs,
            save_path.lower(),
            json_file_name.lower(),
            df_file_name.lower(),
            done,
        )

    # Get html of page with beautifulsoup4
    html = driver.page_source
    print("Feeding html driver to BeautifulSoup.")
    soup = BeautifulSoup(html, "lxml")

    # Find selected language and make sure it's English if available
    # soup = find_language(driver, soup, language)

    # Find page number
    page_number_length, page_number = find_page_number(driver, soup, page_counter)

    # Find job button next siblings with bs4
    jobs, jobs_count, counter, page_counter, page_number, done = get_page_jobs(
        driver,
        soup,
        keyword,
        keyword_file,
        site,
        main_window,
        window_before,
        counter,
        page_counter,
        page_number,
        page_number_length,
        jobs,
        df_old_jobs,
        save_path,
        json_file_name,
        done,
    )

    # Save dict as json file
    jobs = remove_dupe_dicts(jobs)
    print(f"Progress for {keyword} job ads (page {page_number}): {len(jobs)}")
    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)
    done = True

    #     elif done is True:
    #         print('Done is True.')
    #         jobs_count = len(jobs)
    #         try:
    #             page_number = int(soup.find('ul', class_ = 'pagination-list').find(attrs={'aria-current': 'true'}).text)
    #             page_counter = page_number
    #         except:
    #             page_counter = 0

    ############################################### FINAL SAVING AND CLEANUP ###############################################

    # Save dict as json file
    print("Final save.")
    jobs = remove_dupe_dicts(jobs)
    try:
        print(
            f"Progress for {keyword} job ads (page {page_number}): {len(jobs)} / {int(jobs_count)*int(page_number_length)}"
        )
    except:
        print(f"Progress for {keyword} job ads (page {page_number}): {len(jobs)}")

    print(f"Saving dict to json file as {json_file_name} in location {save_path}")
    with open(save_path + json_file_name, "w") as f:
        json.dump(jobs, f)

    # Save jobs to df
    print(f"Saving {keyword} jobs data to df...")
    if jobs:
        df_jobs = pd.DataFrame(jobs)
    elif not jobs:
        print(f"Jobs dict for {keyword} is empty.")
        df_jobs = pd.DataFrame()

    # Save df as csv
    if not df_jobs.empty:
        df_jobs.append(df_old_jobs, ignore_index=True)
        df_jobs = clean_df(df_jobs)
        df_jobs = save_df(
            keyword, df_jobs, save_path, keyword_file.lower(), df_file_name.lower()
        )
    elif df_jobs.empty:
        df_jobs = df_old_jobs

    return jobs, jobs_count, df_jobs, counter, page_counter, done


### Run get_jobs function

In [ ]:
# if __name__ == '__main__':
def run_all_indeed(
    site="Indeed",
    language="English",
    country="Netherlands",
    incognito_enabled=True,
    headless_enabled=False,
    proxy_enabled=False,
    logging_enabled=False,
    print_enabled=True,
):

    # Get keywords
    (
        keywords_list,
        keywords_sector,
        keywords_womenvocc,
        keywords_menvocc,
        keywords_genvsect,
        keywords_oldvocc,
        keywords_youngvocc,
        keywords_agevsect,
    ) = get_keyword_list(print_enabled=print_enabled)
    # remove_list = ['software', 'service', 'storage', 'information', 'sales', 'culture']
    # remove_list = []
    # for k in remove_list:
    #     keywords_list.remove(k)

    # Start driver
    driver = get_driver(select_driver)

    keywords_list = keywords_oldvocc + keywords_youngvocc
    # Start for-loop
    for keyword in keywords_list:
        counter = 1
        recollect = False

        # Check internet connection
        if not is_connected(driver):
            is_connected(driver)

        # Get main info
        (
            keyword_url,
            keyword_file,
            save_path,
            json_file_name,
            df_file_name,
            logs_file_name,
            filemode,
        ) = main_info(keyword, site)
        done = False

        # Set up log
        if logging_enabled is True:
            print("Logging enabled.")
            writer = MyWriter(logs_file_name.lower(), filemode)
            sys.stdout = writer
            logger = logging.getLogger(__name__)
            logging.basicConfig(
                filename=writer.LOGS_PATH + logs_file_name.lower(),
                filemode=filemode,
                level=logging.CRITICAL,
                format="%(asctime)s:%(levelname)s: %(message)s",
            )
        elif logging_enabled is False:
            print("No logging enabled.")

        # Load and merge existing dict and df
        print(f"All files will be saved in folder: {save_path}{json_file_name}")
        jobs, df_old_jobs = load_merge_dict_df(
            keyword, save_path, df_file_name, json_file_name
        )

        # Beging data collection
        print(
            "-" * 20,
            f"BEGINING OF DATA COLLECTION FOR {keyword.upper()} JOB ADS",
            "-" * 20,
        )
        main_window = driver.current_window_handle
        print(f"Main Window: {main_window}")

        # Load and save page_counter
        page_counter_path = (
            save_path + f"{site.lower()}_page_counter_{keyword_file}.txt"
            if recollect is False
            else save_path + f"{site.lower()}_page_counter_done_{keyword_file}.txt"
        )

        if is_non_zero_file(page_counter_path) is True:
            with open(page_counter_path) as f:
                page_counter = int(f.read())
        elif is_non_zero_file(page_counter_path) is False:
            page_counter = 0
        with open(page_counter_path, "w") as f:
            f.write(str(page_counter))

        #         page_counter = 29
        print(f"Starting collection from page number: {page_counter+1}")

        # Start threads
        popup_thread(driver, popup, popup_checker)
        # stale_element_thread(driver, stale_element, stale_element_checker)
        # act_cool_thread(driver, act_cool, act_cool_checker)

        # Get jobs
        jobs, jobs_count, df_jobs, counter, page_counter, done = get_jobs(
            driver,
            keyword.lower(),
            keyword_url,
            keyword_file,
            site,
            language,
            counter,
            page_counter,
            main_window,
            df_old_jobs,
            jobs,
            save_path.lower(),
            json_file_name.lower(),
            df_file_name.lower(),
            done,
        )

        # Save df as csv
        df_jobs = save_df(
            keyword, df_jobs, save_path, keyword_file.lower(), df_file_name.lower()
        )
        print(
            "-" * 20, f"END OF DATA COLLECTION FOR {keyword.upper()} JOB ADS", "-" * 20
        )

    # Close and quit driver
    print(f"Closing driver.")
    driver.close()
    driver.quit()
    # thread_popup_checker.terminate()
    # thread_stale_element_checker.terminate()
    # thread_act_cool_checker.terminate()
    # sys.stdout.close()
    # sys.exit()

    return df_jobs


In [ ]:
while True:
    df_jobs_indeed = run_all_indeed()


In [ ]:
# keyword = 'artsen'
# site = 'Indeed'
# keyword_from_list = True
# site_from_list = False
# df_jobs = post_cleanup(keyword, site)
